<a href="https://colab.research.google.com/github/zyrastory/FjuFinalProject/blob/master/FinLab_Zyra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install finlab > log.txt

In [ ]:
!git clone https://github.com/firekilin/iceWantBeRich.git

Cloning into 'iceWantBeRich'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 21 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (21/21), 340.73 KiB | 4.10 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [ ]:
from iceWantBeRich.common import *

輸入成功!
Due to your status as a free user, the most recent data has been shortened or limited.
Daily usage: 85.5 / 500 MB - price:收盤價
Daily usage: 120.8 / 500 MB - price:成交股數


In [ ]:
from finlab import data
import pandas as pd

close = data.get('price:收盤價')

high = close.rolling(20).max()  # 前 20 天最高價
low = close.rolling(20).min()   # 前 20 天最低價
mean = close.rolling(20).mean() # 前 20 天平均價

print(mean['0050']['2025-01-07'])
print(high['0050']['2025-01-07'])
low['0050']['2025-01-07']

196.8
204.4


192.6

# 主程式

In [ ]:
from finlab import data
import pandas as pd

pd.set_option('display.float_format', '{:.6f}'.format)

close = data.get('price:收盤價')
count = data.get('price:成交股數')

high = close.shift(2).rolling(20).max()  # 前前 20 天最高價
low = close.shift(2).rolling(20).min()   # 前前 20 天最低價
mean = close.shift(2).rolling(20).mean() # 前前 20 天平均價
range_percent = (high - low) / mean * 100  #箱型區間


buyAct = (close.shift(1)>high) & (((close.shift(1)-close.shift(2))/close.shift(2))>0.03) #突破箱型區間，昨日漲超過3%
sellAct = (close.shift(1)<close.shift(2).rolling(5).mean()) #昨天小於5MA


# print(range_percent.index)   # 查看索引
# print(range_percent.columns) # 查看列名
close['type'] = 'close'
high['type'] = 'high'
low['type'] = 'low'
range_percent['type'] = 'range'
#count['type'] = 'count'
buyAct['type'] = 'buyAct'
sellAct['type'] = 'sellAct'



df_combined = pd.concat([close,high,low,range_percent,buyAct,sellAct], axis=0)
df_combined = df_combined.set_index('type', append=True).unstack(level=-1)


#print(df_combined['2024-12-30':])
df_combined = df_combined['2024-11-30':]

allList = close.keys()
allList = allList[:-1]


i = 0
for index,row in df_combined.iterrows():  #按日

  print()
  print('-------------------------------')
  i+=1
  for stock in allList:
     if(row[stock]['buyAct']==1):
      print(stock)
      print(row[stock])

      print()
      print()

  if i == 3:
    break



# 第二隻腳測試

In [ ]:
from finlab import data
import pandas as pd

pd.set_option('display.float_format', '{:.4f}'.format)

close = data.get('price:收盤價')
count = data.get('price:成交股數')

#shift(7) > 突破日
#shift(2) rolling 前天(包含)往前幾天 >> 判斷拉回支撐

high = close.shift(8).rolling(30).max()  # 前前 X 天最高價
low = close.shift(8).rolling(30).min()   # 前前 X 天最低價
mean = close.shift(8).rolling(30).mean() # 前前 X 天平均價
range_percent = (high - low) / mean * 100  #箱型區間


#昨日前5天的前一天 突破箱型 且當天至少漲4-7%  帶量漲破(超過30%)
breakout = (close.shift(7) > high) & (((close.shift(7) - close.shift(8)) / close.shift(8)) > 0.04) & (((close.shift(7) - close.shift(8)) / close.shift(8)) < 0.07) & \
 (count.shift(7)>(count.shift(8).rolling(30).median())*1.3)

#拉回用昨日以前5天判斷  5 4 3 2 1 N N+1  (N 為買賣判斷日 N+1為今日)
#有拉回且不跌破箱型上緣太多  拉回4%以上 且不跌破箱型中心 量縮價跌
pullback = (close.shift(2).rolling(5).min() < close.shift(7)*0.96) & (close.shift(2).rolling(5).min() > mean) & (count.shift(2).rolling(5).mean()<count.shift(7)*0.7)

#支撐  昨日重新突破箱型上緣 為上漲 且量能有出來 (量能小於第一支腳)
support_formed = pullback & (close.shift(1) > close.shift(2)) &(close.shift(1) > high) & (count.shift(1)>count.shift(2).rolling(5).mean()*1.2) &\
 (count.shift(1)<count.shift(7))


#加上判斷震盪 20%以內
second_leg = breakout & support_formed & (range_percent<20) & (count.shift(1).rolling(50).median()>(500*1000))



# print(range_percent.index)   # 查看索引
# print(range_percent.columns) # 查看列名
close['type'] = 'close'
# high['type'] = 'high'
# low['type'] = 'low'
#range_percent['type'] = 'range'
#count['type'] = 'count'
second_leg['type'] = 'second_leg'
#sellAct['type'] = 'sellAct'



df_combined = pd.concat([close,second_leg], axis=0)
df_combined = df_combined.set_index('type', append=True).unstack(level=-1)


#print(df_combined['2024-12-30':])
df_combined = df_combined['2024-01-30':]

allList = close.keys()
allList = allList[:-1]


i = 0

for index,row in df_combined.iterrows():  #按日

  i+=1
  j=0
  buylist=[]
  for stock in allList:
     if(row[stock]['second_leg']==1):
      # print(stock)
      # print(row[stock])
      j+=1
      buylist.append(stock)

      # print()
      # print()

  if(j>0):
    print(index)
    print(j)
    print(buylist)
    print('-------------------------------')
    print()



KeyboardInterrupt: 

# 第二隻腳+列印

In [ ]:
from finlab import data
import pandas as pd

pd.set_option('display.float_format', '{:.4f}'.format)

close = data.get('price:收盤價')
count = data.get('price:成交股數')

close = close['2022-01-01':]

#shift(7) > 突破日
#shift(2) rolling 前天(包含)往前幾天 >> 判斷拉回支撐

high = close.shift(8).rolling(40).max()  # 前前 X 天最高價
low = close.shift(8).rolling(40).min()   # 前前 X 天最低價
mean = close.shift(8).rolling(40).mean() # 前前 X 天平均價
range_percent = (high - low) / mean * 100  #箱型區間
sellAct = (close.shift(1)<close.shift(2).rolling(5).mean()) #昨天小於*MA


#昨日前5天的前一天 突破箱型 且當天至少漲3-7%  帶量漲破(超過30%)
breakout = (close.shift(7) > high) & (((close.shift(7) - close.shift(8)) / close.shift(8)) > 0.03) & (((close.shift(7) - close.shift(8)) / close.shift(8)) < 0.07) & \
 (count.shift(7)>(count.shift(8).rolling(40).median())*1.3)

#拉回用昨日以前5天判斷  5 4 3 2 1 N N+1  (N 為買賣判斷日 N+1為今日)
#有拉回且不跌破箱型上緣太多  拉回5%以上 且不跌破箱型中心 量縮價跌
pullback = (close.shift(2).rolling(5).min() < close.shift(7)*0.95) & (close.shift(2).rolling(5).min() > mean) & (count.shift(2).rolling(5).mean()<count.shift(7)*0.7)

#支撐  昨日重新突破箱型上緣 為上漲 且量能有出來 (量能小於第一支腳)
support_formed = pullback & (close.shift(1) > close.shift(2)) &(close.shift(1) > high) & (count.shift(1)>count.shift(2).rolling(5).mean()*1.2) &\
 (count.shift(1)<count.shift(7))


#加上判斷震盪 N%以內
second_leg = breakout & support_formed & (range_percent<25) & (count.shift(1).rolling(50).median()>(500*1000))



# print(range_percent.index)   # 查看索引
# print(range_percent.columns) # 查看列名
close['type'] = 'close'
# high['type'] = 'high'
# low['type'] = 'low'
#range_percent['type'] = 'range'
#count['type'] = 'count'
second_leg['type'] = 'second_leg'
sellAct['type'] = 'sellAct'



df_combined = pd.concat([close,second_leg,sellAct], axis=0)
df_combined = df_combined.set_index('type', append=True).unstack(level=-1)


allList = close.keys()
allList = allList[:-1]


i = 0
nowDict = {}
win = 0
lose = 0
avg = 0

for index,row in df_combined.iterrows():  #按日

  i+=1
  j=0
  buylist=[]
  for stock in allList:
     if(stock not in nowDict and row[stock]['second_leg']==1):
      #if(stock not in nowDict):
        # print(stock)
        # print(row[stock])
        # j+=1
        # buylist.append(stock)
      nowDict[stock] = {"buy_price": row[stock]['close'], "buy_date": index}
     elif(stock in nowDict and row[stock]['sellAct']==1):
      profit = ((row[stock]['close']-nowDict[stock]['buy_price'])/nowDict[stock]['buy_price'])*100
      profit = round(profit, 2)
      avg+=profit
      print(f"股票代號: {stock}, 買進價{nowDict[stock]['buy_price']}, 買進日期：{nowDict[stock]['buy_date']}, 賣出日期: {index}, 賣出金額: {row[stock]['close']}, 利潤(未扣除手續費): {profit}%")
      if(profit>0):
        win+=1
      else:
        lose+=1
      print()
      del nowDict[stock]


      # print()
      # print()
avg/=(win+lose)
avg = round(avg, 2)
print(f"總次數:{win+lose}次, 獲利:{win}次, 虧損:{lose}次, 平均獲利/虧損: {avg}")
  # if(j>0):
  #   print(index)
  #   print(j)
  #   print(buylist)
  #   print('-------------------------------')
  #   print()



# 提高箱型判斷條件

In [ ]:
from finlab import data
import pandas as pd

pd.set_option('display.float_format', '{:.4f}'.format)

close = data.get('price:收盤價')
count = data.get('price:成交股數')

close = close['2022-11-01':'2024-12-31']
count = count['2022-11-01':'2024-12-31']

#shift(7) > 突破日
#shift(2) rolling 前天(包含)往前幾天 >> 判斷拉回支撐

high = close.shift(8).rolling(40).max()*0.99  # 前前 X 天最高價
low = close.shift(8).rolling(40).min()*1.1   # 前前 X 天最低價
mean = close.shift(8).rolling(40).mean() # 前前 X 天平均價

maOld = close.shift(28).rolling(20).mean()
maNew = close.shift(8).rolling(20).mean()


hits_high = (close.shift(8)>high*0.98).rolling(40).sum()  # 接近高點的次數
hits_low = (close.shift(8)<low*1.02).rolling(40).sum()   # 接近低點的次數

isbox = (hits_high>5) & (((high - low) / mean * 100)<25) & (maNew>maOld*1.03)

# 計算每日漲跌幅（%）
daily_return = close.pct_change() * 100

#range_percent = (high - low) / mean * 100  #箱型區間
sellAct = ((daily_return+daily_return.shift(1))>16) | (daily_return.rolling(5).sum()>30)  #停利狀況


#昨日前5天的前一天 突破箱型 且當天至少漲2-6%  帶量漲破(超過150%)
breakout = (close.shift(7) > high) & (((close.shift(7) - close.shift(8)) / close.shift(8)) > 0.02) & (((close.shift(7) - close.shift(8)) / close.shift(8)) < 0.06) & \
 (count.shift(7)>(count.shift(8).rolling(40).mean())*2.49)

#拉回用昨日以前5天判斷  5 4 3 2 1 N N+1  (N 為買賣判斷日 N+1為今日)
#有拉回且不跌破箱型上緣太多  拉回3%以上 且不跌破箱型中心 量縮價跌
pullback = (close.shift(2).rolling(5).min() < close.shift(7)*0.97) & (close.shift(2).rolling(5).min() > mean*0.985) & (count.shift(2).rolling(5).mean()<count.shift(7)*0.6)

#support_formed = pullback & (close.shift(1)>close.shift(2)) &(close.shift(1) > high) #& (count.shift(1)>count.shift(2).rolling(5).mean()*1.2)
#測試 量能增加且不漲太多才進場
#support_formed = pullback & (close.shift(1)>close.shift(2).rolling(5).mean()*1.02) & (close.shift(1)<close.shift(2).rolling(5).max()*1.05) & (count.shift(1)>count.shift(2).rolling(5).mean()*1.2)
support_formed = pullback & (close.shift(1)>close.shift(2)*1.02) & (close.shift(1)<close.shift(2)*1.04) & (count.shift(1)>count.shift(2).rolling(5).mean()*1.2)

#排除太小型股 500張以下
second_leg = breakout & support_formed & (count.shift(1).rolling(40).median()>(500*1000))

second_leg = second_leg & isbox


# print(range_percent.index)   # 查看索引
# print(range_percent.columns) # 查看列名
close['type'] = 'close'
# high['type'] = 'high'
low['type'] = 'stop_loss'
#range_percent['type'] = 'range'
#count['type'] = 'count'
second_leg['type'] = 'second_leg'
sellAct['type'] = 'sellAct'



df_combined = pd.concat([close,second_leg,low,sellAct], axis=0)
df_combined = df_combined.set_index('type', append=True).unstack(level=-1)

print(df_combined.shape[0])


allList = close.keys()
allList = allList[:-1]


i = 0
nowDict = {}
win = 0
lose = 0
allprofit = 0
hSet = set()
printList = []
for int_index, (index, row) in enumerate(df_combined.iterrows()):  #按日
  hTmp = set()
  for stock in allList:
     if(stock not in nowDict and row[stock]['second_leg']==1):
      nowDict[stock] = {"buy_price": row[stock]['close'], "buy_date": index, "stop_loss_price": row[stock]['stop_loss']*1.01, "int_index": int_index }
      hTmp.add(stock)

     #elif(stock in nowDict and row[stock]['sellAct']==1):
     elif(stock in nowDict):
      if(row[stock]['sellAct']==1):
        if(stock in hSet):
          continue
        profit = ((row[stock]['close']-nowDict[stock]['buy_price'])/nowDict[stock]['buy_price'])*100
        profit = round(profit, 2)
        allprofit+=profit
        printList.append(f"股票代號: {stock}, 買進價{nowDict[stock]['buy_price']}, 買進日期：{nowDict[stock]['buy_date']}, 賣出日期: {index}, 賣出金額: {row[stock]['close']}, 利潤(未扣除手續費): {profit}%")
        if(profit>0):
          win+=1
        else:
          lose+=1
        #print()
        del nowDict[stock]
      elif(nowDict[stock]['stop_loss_price']>row[stock]['close']):
        if(stock in hSet):
          continue
        profit = ((row[stock]['close']-nowDict[stock]['buy_price'])/nowDict[stock]['buy_price'])*100
        profit = round(profit, 2)
        allprofit+=profit
        printList.append(f"股票代號: {stock}, 買進價{nowDict[stock]['buy_price']}, 買進日期：{nowDict[stock]['buy_date']}, 賣出日期: {index}, 賣出金額: {row[stock]['close']}, 利潤(未扣除手續費): {profit}% 備註:跌破箱型中下緣 {nowDict[stock]['stop_loss_price']}")
        if(profit>0):
          win+=1
        else:
          lose+=1
        del nowDict[stock]
      elif(int_index>(nowDict[stock]['int_index']+40)): #不持倉超過2個月
        profit = ((row[stock]['close']-nowDict[stock]['buy_price'])/nowDict[stock]['buy_price'])*100
        profit = round(profit, 2)
        allprofit+=profit
        printList.append(f"股票代號: {stock}, 買進價{nowDict[stock]['buy_price']}, 買進日期：{nowDict[stock]['buy_date']}, 賣出日期: {index}, 賣出金額: {row[stock]['close']}, 利潤(未扣除手續費): {profit}% 備註:持有天數超過")
        if(profit>0):
          win+=1
        else:
          lose+=1
        del nowDict[stock]


  hSet = hTmp

      # print()
      # print()
avg = 0
if((win+lose)>0):
  avg=allprofit/(win+lose)
  avg = round(avg, 2)
print(f"總次數:{win+lose}次, 獲利:{win}次, 虧損:{lose}次, 平均每次獲利/虧損: {avg}, 總報酬率: {allprofit}")

for probj in printList:
  print(probj)


請從 https://ai.finlab.tw/api_token 複製驗證碼: 



KeyboardInterrupt: Interrupted by user

# 使用不同箱型型態去計算

In [2]:
from finlab import data
import pandas as pd

pd.set_option('display.float_format', '{:.4f}'.format)

close = data.get('price:收盤價')
count = data.get('price:成交股數')

close = close['2021-11-01':'2022-12-31']
count = count['2021-11-01':'2022-12-31']

def getBuySignal(boxDate,chkDate):

  high = close.shift(chkDate+3).rolling(boxDate).max()*0.9  # 前前 X 天最高價
  low = close.shift(chkDate+3).rolling(boxDate).min()*1.1   # 前前 X 天最低價
  mean = close.shift(chkDate+3).rolling(boxDate).mean() # 前前 X 天平均價

  maOld = close.shift(chkDate+3+boxDate//2).rolling(boxDate//2).mean()
  maNew = close.shift(chkDate+3).rolling(boxDate//2).mean()


  hits_high = (close.shift(chkDate+3)>high*0.98).rolling(boxDate).sum()  # 接近高點的次數
  hits_low = (close.shift(chkDate+3)<low*1.02).rolling(boxDate).sum()   # 接近低點的次數

  isbox = (hits_high<5) & (((high-low)/mean*100)<20) & (maNew>maOld*1.03) #& (mean<close.shift(chkDate+3+boxDate).rolling(boxDate).mean()*1.3)  #測試 希望區間前段也不要太高的漲幅???

  #昨日前5天的前一天 (6天前) 突破箱型 且當天小於6%  帶量漲破(超過100%)
  breakout = (close.shift(chkDate+2) > high) & (((close.shift(chkDate+2) - close.shift(chkDate+3)) / close.shift(chkDate+3)) < 0.06) & \
  (count.shift(chkDate+2)>(count.shift(chkDate+3).rolling(boxDate).mean())*2) & (count.shift(chkDate+2)>(count.shift(chkDate+3).rolling(boxDate).mean())*5)

  #拉回用昨日以前5天判斷  5 4 3 2 1 N N+1  (N 為買賣判斷日 N+1為今日)
  #有拉回且不跌破箱型上緣太多  拉回2%以上 且不跌破箱型中心 價跌或是持平          僅剩當天7成量?? 量縮??
  #pullback = (close.shift(2).rolling(chkDate).min() < close.shift(chkDate+2)*0.98) & (close.shift(2).rolling(chkDate).min() > mean*0.985) #& (count.shift(2).rolling(chkDate).mean()<count.shift(7)*0.7)

  #20250111 換策略 找部分拉回或是持平(相對突破日而言) 強勢股  持續放量
  pullback = (close.shift(2).rolling(chkDate).min()>close.shift(chkDate+2)*0.85) & (close.shift(2).rolling(chkDate).max()<close.shift(chkDate+2)*1.15) & (count.shift(2).rolling(chkDate).mean()>count.shift(7)*0.5)

  #測試 昨日不漲跌太多考慮隔日進場
  support_formed = pullback & (close.shift(1)>close.shift(2)*0.96) & (close.shift(1)<close.shift(2)*1.04) & (count.shift(1)>count.shift(7)*0.5)

  #排除太小型股 2000張以下
  second_leg = breakout & support_formed & (count.shift(1).rolling(40).mean()>(1999*1000))

  second_leg = second_leg & isbox

  return second_leg

def combine_signals(*signals):
    combined = pd.concat(signals, axis=0)
    combined['type'] = 'buySignals'
    return combined

buySignal1 = getBuySignal(40,3)
buySignal2 = getBuySignal(30,3)
buySignal3 = getBuySignal(20,3)
buySignals = buySignal1 | buySignal2 | buySignal3


# 計算每日漲跌幅（%）
daily_return = close.pct_change() * 100

#range_percent = (high - low) / mean * 100  #箱型區間
#sellAct = ((daily_return+daily_return.shift(1))>16) | (daily_return.rolling(4).sum()>16)  #快速停利狀況
sellAct = ((daily_return.shift(1)+daily_return.shift(2))>16) | (daily_return.shift(1).rolling(3).sum()>18)  #快速停利狀況

close['type'] = 'close'
#low['type'] = 'stop_loss'
#second_leg['type'] = 'second_leg'
sellAct['type'] = 'sellAct'
buySignals['type'] = 'buySignals'



df_combined = pd.concat([close,buySignals,sellAct,daily_return], axis=0)
df_combined = df_combined.set_index('type', append=True).unstack(level=-1)

print(df_combined.shape[0])


allList = close.keys()
allList = allList[:-1]


i = 0
nowDict = {}
win = 0
lose = 0
allprofit = 0

printList = []

previous_row = None  # 用來保存上一行的資料

for int_index, (index, row) in enumerate(df_combined.iterrows()):  #按日

  for stock in allList:
     if(stock not in nowDict and row[stock]['buySignals']==1):
      if(row[stock]['close']<previous_row[stock]['close']*1.05 and row[stock]['close']>previous_row[stock]['close']*0.94): #比起昨日漲跌5%以上不接
        nowDict[stock] = {"buy_price": row[stock]['close'], "buy_date": index, "stop_loss_price": row[stock]['close']*0.85, "int_index": int_index }

     elif(stock in nowDict):
      if(row[stock]['sellAct']==1 and int_index>(nowDict[stock]['int_index']+3)): #至少持股3天以上
        profit = ((row[stock]['close']-nowDict[stock]['buy_price'])/nowDict[stock]['buy_price'])*100
        profit = round(profit, 2)
        allprofit+=profit
        printList.append(f"股票代號: {stock}, 買進價{nowDict[stock]['buy_price']}, 買進日期：{nowDict[stock]['buy_date']}, 賣出日期: {index}, 賣出金額: {row[stock]['close']}, 利潤(未扣除手續費): {profit}% 備註:短期暴利停利")
        if(profit>0):
          win+=1
        else:
          lose+=1
        #print()
        del nowDict[stock]
      elif(nowDict[stock]['stop_loss_price']>row[stock]['close']):
        profit = ((row[stock]['close']-nowDict[stock]['buy_price'])/nowDict[stock]['buy_price'])*100
        profit = round(profit, 2)
        allprofit+=profit
        printList.append(f"股票代號: {stock}, 買進價{nowDict[stock]['buy_price']}, 買進日期：{nowDict[stock]['buy_date']}, 賣出日期: {index}, 賣出金額: {row[stock]['close']}, 利潤(未扣除手續費): {profit}% 備註:跌破停損點位 {nowDict[stock]['stop_loss_price']}")
        if(profit>0):
          win+=1
        else:
          lose+=1
        del nowDict[stock]
      elif(row[stock]['close']>nowDict[stock]['buy_price']*1.2): #最大利潤 20%左右
        profit = ((row[stock]['close']-nowDict[stock]['buy_price'])/nowDict[stock]['buy_price'])*100
        profit = round(profit, 2)
        allprofit+=profit
        printList.append(f"股票代號: {stock}, 買進價{nowDict[stock]['buy_price']}, 買進日期：{nowDict[stock]['buy_date']}, 賣出日期: {index}, 賣出金額: {row[stock]['close']}, 利潤(未扣除手續費): {profit}% 備註:停利(與最初價)")
        if(profit>0):
          win+=1
        else:
          lose+=1
        del nowDict[stock]
      elif(int_index>(nowDict[stock]['int_index']+10) and row[stock]['close']>nowDict[stock]['buy_price']*1.1): #持倉超過半個月 且 利潤N%左右
        profit = ((row[stock]['close']-nowDict[stock]['buy_price'])/nowDict[stock]['buy_price'])*100
        profit = round(profit, 2)
        allprofit+=profit
        printList.append(f"股票代號: {stock}, 買進價{nowDict[stock]['buy_price']}, 買進日期：{nowDict[stock]['buy_date']}, 賣出日期: {index}, 賣出金額: {row[stock]['close']}, 利潤(未扣除手續費): {profit}% 備註:持有天數超過半個月 且 獲得部分利潤")
        if(profit>0):
          win+=1
        else:
          lose+=1
        del nowDict[stock]

      elif(int_index>(nowDict[stock]['int_index']+20)): #不持倉超過一個月
        profit = ((row[stock]['close']-nowDict[stock]['buy_price'])/nowDict[stock]['buy_price'])*100
        profit = round(profit, 2)
        allprofit+=profit
        printList.append(f"股票代號: {stock}, 買進價{nowDict[stock]['buy_price']}, 買進日期：{nowDict[stock]['buy_date']}, 賣出日期: {index}, 賣出金額: {row[stock]['close']}, 利潤(未扣除手續費): {profit}% 備註:持有天數超過最久天期")
        if(profit>0):
          win+=1
        else:
          lose+=1
        del nowDict[stock]

  previous_row = row

avg = 0
if((win+lose)>0):
  avg=allprofit/(win+lose)
  avg = round(avg, 2)
print(f"總次數:{win+lose}次, 獲利:{win}次, 虧損:{lose}次, 平均每次獲利/虧損: {avg}, 總報酬率: {allprofit}")

for probj in printList:
  print(probj)

print('-----------------------')
print(nowDict)

print(f"總次數:{win+lose}次, 獲利:{win}次, 虧損:{lose}次, 平均每次獲利/虧損: {avg}, 總報酬率: {allprofit}")


請從 https://ai.finlab.tw/api_token 複製驗證碼: 



KeyboardInterrupt: Interrupted by user

# 調整箱型 改為以量為主

In [33]:
from finlab import data
import pandas as pd

pd.set_option('display.float_format', '{:.4f}'.format)

close = data.get('price:收盤價')
count = data.get('price:成交股數')

close = close['2021-11-01':'2022-12-31']
count = count['2021-11-01':'2022-12-31']

# 計算每日漲跌幅（%）
daily_return = close.pct_change() * 100

def getBuySignal(boxDate,chkDate):

  high = close.shift(chkDate+3).rolling(boxDate).max()*0.9  # 前前 X 天最高價
  low = close.shift(chkDate+3).rolling(boxDate).min()*1.1   # 前前 X 天最低價
  mean = close.shift(chkDate+3).rolling(boxDate).mean() # 前前 X 天平均價

  maOld = close.shift(chkDate+3+boxDate).rolling(boxDate).mean()
  maNew = close.shift(chkDate+3).rolling(boxDate).mean()

  hits_high = (close.shift(chkDate+3)>high*0.98).rolling(boxDate).sum()  # 接近高點的次數
  hits_low = (close.shift(chkDate+3)<low*1.02).rolling(boxDate).sum()   # 接近低點的次數

  #high_act = (daily_return.shift(chkDate+3)>8).rolling(boxDate).sum()
  low_act = (daily_return.shift(chkDate+3)<-7).rolling(boxDate).sum()

  isbox = (hits_high<10) & (((high-low)/mean*100)<25) & (maNew>maOld*1.05) & (maNew<maOld*1.2)  #測試 希望區間前段也不要太高的漲幅???
  #isbox = isbox & (low_act==0)

  #帶量漲破(超過150%)  #(close.shift(chkDate+2) > high) &
  #breakout = (((close.shift(chkDate+2)-close.shift(chkDate+3))/close.shift(chkDate+3))<0.06) & (((close.shift(chkDate+2)-close.shift(chkDate+3))/close.shift(chkDate+3))>0) & \
  breakout = (daily_return.shift(chkDate+2)<6) & (daily_return.shift(chkDate+2)>0) & \
  (count.shift(chkDate+2)>(count.shift(chkDate+3).rolling(boxDate).mean())*2.5) & (count.shift(chkDate+2)>(count.shift(chkDate+3).rolling(boxDate).mean())*5)


  #20250111 換策略 找部分拉回或是持平(相對突破日而言) 強勢股  持續放量
  pullback = (close.shift(2).rolling(chkDate).min()>close.shift(chkDate+2)*0.85) & (close.shift(2).rolling(chkDate).max()<close.shift(chkDate+2)*1.15) & (count.shift(2).rolling(chkDate).mean()>count.shift(7)*0.5)

  #測試 昨日不漲跌太多考慮隔日進場
  support_formed = pullback & (close.shift(1)>close.shift(2)*0.96) & (close.shift(1)<close.shift(2)*1.04) & (count.shift(1)>count.shift(7)*0.5)

  #排除太小型股 2000張以下
  second_leg = breakout & support_formed & (count.shift(1).rolling(40).mean()>(1999*1000))

  second_leg = second_leg & isbox

  return second_leg

def combine_signals(*signals):
    combined = pd.concat(signals, axis=0)
    combined['type'] = 'buySignals'
    return combined

buySignal1 = getBuySignal(40,3)
buySignal2 = getBuySignal(30,3)
#buySignal3 = getBuySignal(20,3)
buySignals = buySignal1 | buySignal2 #| buySignal3




sellAct = ((daily_return.shift(1)+daily_return.shift(2))>16) | (daily_return.shift(1).rolling(3).sum()>18)  #快速停利狀況

close['type'] = 'close'
sellAct['type'] = 'sellAct'
buySignals['type'] = 'buySignals'



df_combined = pd.concat([close,buySignals,sellAct,daily_return], axis=0)
df_combined = df_combined.set_index('type', append=True).unstack(level=-1)

print(df_combined.shape[0])


allList = close.keys()
allList = allList[:-1]


i = 0
nowDict = {}
win = 0
lose = 0
allprofit = 0

printList = []

previous_row = None  # 用來保存上一行的資料

for int_index, (index, row) in enumerate(df_combined.iterrows()):  #按日

  for stock in allList:
     if(stock not in nowDict and row[stock]['buySignals']==1):
      if(row[stock]['close']<previous_row[stock]['close']*1.05 and row[stock]['close']>previous_row[stock]['close']*0.94): #比起昨日漲跌5%以上不接
        nowDict[stock] = {"buy_price": row[stock]['close'], "buy_date": index, "stop_loss_price": row[stock]['close']*0.85, "int_index": int_index }

     elif(stock in nowDict):
      if(row[stock]['sellAct']==1 and int_index>(nowDict[stock]['int_index']+3)): #至少持股3天以上
        profit = ((row[stock]['close']-nowDict[stock]['buy_price'])/nowDict[stock]['buy_price'])*100
        profit = round(profit, 2)
        allprofit+=profit
        printList.append(f"股票代號: {stock}, 買進價{nowDict[stock]['buy_price']}, 買進日期：{nowDict[stock]['buy_date']}, 賣出日期: {index}, 賣出金額: {row[stock]['close']}, 利潤(未扣除手續費): {profit}% 備註:短期暴利停利")
        if(profit>0):
          win+=1
        else:
          lose+=1
        #print()
        del nowDict[stock]
      elif(nowDict[stock]['stop_loss_price']>row[stock]['close']):
        profit = ((row[stock]['close']-nowDict[stock]['buy_price'])/nowDict[stock]['buy_price'])*100
        profit = round(profit, 2)
        allprofit+=profit
        printList.append(f"股票代號: {stock}, 買進價{nowDict[stock]['buy_price']}, 買進日期：{nowDict[stock]['buy_date']}, 賣出日期: {index}, 賣出金額: {row[stock]['close']}, 利潤(未扣除手續費): {profit}% 備註:跌破停損點位 {nowDict[stock]['stop_loss_price']}")
        if(profit>0):
          win+=1
        else:
          lose+=1
        del nowDict[stock]
      elif(row[stock]['close']>nowDict[stock]['buy_price']*1.2): #最大利潤 20%左右
        profit = ((row[stock]['close']-nowDict[stock]['buy_price'])/nowDict[stock]['buy_price'])*100
        profit = round(profit, 2)
        allprofit+=profit
        printList.append(f"股票代號: {stock}, 買進價{nowDict[stock]['buy_price']}, 買進日期：{nowDict[stock]['buy_date']}, 賣出日期: {index}, 賣出金額: {row[stock]['close']}, 利潤(未扣除手續費): {profit}% 備註:停利(與最初價)")
        if(profit>0):
          win+=1
        else:
          lose+=1
        del nowDict[stock]
      elif(int_index>(nowDict[stock]['int_index']+10) and row[stock]['close']>nowDict[stock]['buy_price']*1.1): #持倉超過半個月 且 利潤N%左右
        profit = ((row[stock]['close']-nowDict[stock]['buy_price'])/nowDict[stock]['buy_price'])*100
        profit = round(profit, 2)
        allprofit+=profit
        printList.append(f"股票代號: {stock}, 買進價{nowDict[stock]['buy_price']}, 買進日期：{nowDict[stock]['buy_date']}, 賣出日期: {index}, 賣出金額: {row[stock]['close']}, 利潤(未扣除手續費): {profit}% 備註:持有天數超過半個月 且 獲得部分利潤")
        if(profit>0):
          win+=1
        else:
          lose+=1
        del nowDict[stock]

      elif(int_index>(nowDict[stock]['int_index']+20)): #不持倉超過一個月
        profit = ((row[stock]['close']-nowDict[stock]['buy_price'])/nowDict[stock]['buy_price'])*100
        profit = round(profit, 2)
        allprofit+=profit
        printList.append(f"股票代號: {stock}, 買進價{nowDict[stock]['buy_price']}, 買進日期：{nowDict[stock]['buy_date']}, 賣出日期: {index}, 賣出金額: {row[stock]['close']}, 利潤(未扣除手續費): {profit}% 備註:持有天數超過最久天期")
        if(profit>0):
          win+=1
        else:
          lose+=1
        del nowDict[stock]

  previous_row = row

avg = 0
if((win+lose)>0):
  avg=allprofit/(win+lose)
  avg = round(avg, 2)
print(f"總次數:{win+lose}次, 獲利:{win}次, 虧損:{lose}次, 平均每次獲利/虧損: {avg}, 總報酬率: {allprofit}")

for probj in printList:
  print(probj)

print('-----------------------')
print(nowDict)

print(f"總次數:{win+lose}次, 獲利:{win}次, 虧損:{lose}次, 平均每次獲利/虧損: {avg}, 總報酬率: {allprofit}")


<ipython-input-33-e086fb27670f>:13: FutureWarning: The default fill_method='pad' in FinlabDataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_return = close.pct_change() * 100


290
總次數:1次, 獲利:0次, 虧損:1次, 平均每次獲利/虧損: -4.74, 總報酬率: -4.74
股票代號: 6235, 買進價39.0, 買進日期：2022-03-30 00:00:00, 賣出日期: 2022-05-03 00:00:00, 賣出金額: 37.15, 利潤(未扣除手續費): -4.74% 備註:持有天數超過最久天期
-----------------------
{}
總次數:1次, 獲利:0次, 虧損:1次, 平均每次獲利/虧損: -4.74, 總報酬率: -4.74


In [6]:
from finlab import data
import pandas as pd

pd.set_option('display.float_format', '{:.4f}'.format)

close = data.get('price:收盤價')
count = data.get('price:成交股數')

close = close['2021-11-01':'2022-12-31']
count = count['2021-11-01':'2022-12-31']

# 計算每日漲跌幅（%）
daily_return = close.pct_change() * 100

daily_return

<ipython-input-6-e42ad201a8c9>:13: FutureWarning: The default fill_method='pad' in FinlabDataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_return = close.pct_change() * 100


symbol,0015,0050,0051,0052,0053,0054,0055,0056,0057,0058,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2021-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-02,NaN,-0.1832,-0.5119,-0.2002,0.1503,0.3537,-0.1671,-0.0951,0.2668,NaN,...,-3.1532,-3.8082,0.0000,-0.4329,-1.4354,-0.7335,-1.3928,-3.0435,0.3617,-0.6289
2021-11-03,NaN,0.2569,0.8576,0.4414,0.8252,0.2884,-0.2510,0.7612,0.1597,NaN,...,2.0930,2.0528,0.6689,0.0000,-1.9417,0.8621,-1.1299,0.4484,0.5405,3.1646
2021-11-04,NaN,-0.3294,-0.0850,-0.4395,-0.4464,0.0000,0.0419,0.2833,0.2657,NaN,...,0.9112,-1.5805,0.3322,0.8696,0.0000,-1.2210,-1.7143,-1.3393,0.0000,-1.5337
2021-11-05,NaN,1.4322,0.7660,1.9262,1.3453,0.1917,0.0000,0.2825,0.5829,NaN,...,1.3544,-2.1898,0.6623,-1.2931,0.9901,-0.3708,0.2907,0.0000,0.5376,-0.6231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,NaN,0.3613,0.4873,0.1068,0.0948,0.0000,0.1392,-0.0781,0.0633,NaN,...,-1.2165,0.0000,0.4098,0.0000,-0.9132,0.2990,-0.7299,-3.0371,1.3462,-0.5797
2022-12-27,NaN,0.4500,0.3880,0.4800,0.3788,0.0000,0.1854,0.1955,0.5057,NaN,...,-0.2463,0.5727,0.4082,0.0000,0.0000,0.0000,-0.2451,2.0882,0.7590,-0.2915
2022-12-28,NaN,-1.4337,-0.5797,-1.6985,-1.8868,0.0000,-0.4163,-0.9754,-1.7610,NaN,...,0.9877,-1.7084,-0.8130,0.2604,0.4608,-1.6393,-0.9828,4.4318,-0.5650,0.2924


In [19]:
from finlab import data
import pandas as pd

pd.set_option('display.float_format', '{:.4f}'.format)

close = data.get('price:收盤價')
count = data.get('price:成交股數')

close = close['2021-11-01':'2022-12-31']
count = count['2021-11-01':'2022-12-31']

# 計算每日漲跌幅（%）
daily_return = close.pct_change() * 100

high_act = (daily_return.shift(5)>7).rolling(40).sum()
high_act['6235']['2022-03-25':]

<ipython-input-19-b65df2ac106d>:13: FutureWarning: The default fill_method='pad' in FinlabDataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_return = close.pct_change() * 100


,6235
date,
2022-03-25,2.0000
2022-03-28,2.0000
2022-03-29,3.0000
2022-03-30,3.0000
2022-03-31,3.0000
...,...
2022-12-26,1.0000
2022-12-27,1.0000
2022-12-28,1.0000


# 加上持股筆數 手續費等判斷

In [ ]:
from finlab import data
import pandas as pd

pd.set_option('display.float_format', '{:.4f}'.format)

close = data.get('price:收盤價')
count = data.get('price:成交股數')

close = close['2022-11-01':'2023-12-31']
count = count['2022-11-01':'2023-12-31']

stockSaveList =  pd.DataFrame({"代碼": [''], "買進股價": [''], "股數": [''], "購買金額": [''],"購買時間":[''],"賣出股價": [''],"賣出金額":[''],"賣出時間":[''],"淨值":[''],"獲利趴數":[''],"備註":['']}) #儲存清單
stockList = [] #回測清單



def getBuySignal(boxDate,chkDate):

  high = close.shift(chkDate+3).rolling(boxDate).max()*0.9  # 前前 X 天最高價
  low = close.shift(chkDate+3).rolling(boxDate).min()*1.1   # 前前 X 天最低價
  mean = close.shift(chkDate+3).rolling(boxDate).mean() # 前前 X 天平均價

  maOld = close.shift(chkDate+3+boxDate//2).rolling(boxDate//2).mean()
  maNew = close.shift(chkDate+3).rolling(boxDate//2).mean()


  hits_high = (close.shift(chkDate+3)>high*0.98).rolling(boxDate).sum()  # 接近高點的次數
  hits_low = (close.shift(chkDate+3)<low*1.02).rolling(boxDate).sum()   # 接近低點的次數

  isbox = (hits_high<5) & (((high-low)/mean*100)<25) & (maNew>maOld*1.03) #& (mean<close.shift(chkDate+3+boxDate).rolling(boxDate).mean()*1.3)  #測試 希望區間前段也不要太高的漲幅???

  #昨日前5天的前一天 (6天前) 突破箱型 且當天小於6%  帶量漲破(超過100%)
  breakout = (close.shift(chkDate+2) > high) & (((close.shift(chkDate+2) - close.shift(chkDate+3)) / close.shift(chkDate+3)) < 0.06) & \
  (count.shift(chkDate+2)>(count.shift(chkDate+3).rolling(boxDate).mean())*2) & (count.shift(chkDate+2)>(count.shift(chkDate+3).rolling(boxDate).mean())*5)

  #拉回用昨日以前5天判斷  5 4 3 2 1 N N+1  (N 為買賣判斷日 N+1為今日)
  #有拉回且不跌破箱型上緣太多  拉回2%以上 且不跌破箱型中心 價跌或是持平          僅剩當天7成量?? 量縮??
  #pullback = (close.shift(2).rolling(chkDate).min() < close.shift(chkDate+2)*0.98) & (close.shift(2).rolling(chkDate).min() > mean*0.985) #& (count.shift(2).rolling(chkDate).mean()<count.shift(7)*0.7)

  #20250111 換策略 找部分拉回或是持平(相對突破日而言) 強勢股  持續放量
  pullback = (close.shift(2).rolling(chkDate).min()>close.shift(chkDate+2)*0.85) & (close.shift(2).rolling(chkDate).max()<close.shift(chkDate+2)*1.15) & (count.shift(2).rolling(chkDate).mean()>count.shift(7)*0.5)

  #測試 昨日不漲跌太多考慮隔日進場
  support_formed = pullback & (close.shift(1)>close.shift(2)*0.96) & (close.shift(1)<close.shift(2)*1.04) & (count.shift(1)>count.shift(7)*0.5)

  #排除太小型股 2000張以下
  second_leg = breakout & support_formed & (count.shift(1).rolling(40).mean()>(1999*1000))

  second_leg = second_leg & isbox

  return second_leg

def combine_signals(*signals):
    combined = pd.concat(signals, axis=0)
    combined['type'] = 'buySignals'
    return combined


# 得手續費
HANDING_FEE_RATE = (0.1425*0.01);
def getHandingFee(amount):
  rs = round(amount * HANDING_FEE_RATE);
  if rs<20:
    rs=20;
  return rs;

# 取得交易稅
TAX_RATE = (0.3*0.01)
def getTax(amount):
  return round(amount * TAX_RATE);

# 取得出售淨值
def getCleanPrice(amount):
  return round(amount - getHandingFee(amount) - getTax(amount));

# 以價格取得股數
def getUnit(price,totalPrice):
  avg =round(totalPrice / price)
  while avg * price + getHandingFee(avg * price) > totalPrice:
    avg -= 1;
  return avg;

#購買
def buyStock(stockList,id,close,date,index):
  for i in stockList :
    if i['id'] == "":
      i['id'] = id
      nowPrice = close
      i['stockPrice'] = nowPrice
      i['unit'] = getUnit(nowPrice, i['balance'])
      i['buyPrice'] = nowPrice * i['unit']
      i['balance'] = i['balance'] - i['buyPrice'] - getHandingFee(i['buyPrice'])
      i['buyDate'] = date
      i['buyIndex'] = index
      i['stop_loss_price'] = close * 0.85
      return stockList
  return stockList

#賣出
def sellStock(stockSaveList, stockList, id, close, date, memo):
  for i in stockList:
    if i['id'] == id:
        nowPrice = close
        i['sellPrice'] = getCleanPrice(nowPrice * i['unit'])
        i['sellDate'] = date
        i['balance'] += i['sellPrice']

        stockSaveList.loc[len(stockSaveList)] = [
            id,
            str(i['stockPrice']),
            str(i['unit']),
            str(i['buyPrice']),
            i['buyDate'],
            str(close),
            str(i['sellPrice']),
            i['sellDate'],
            str(i['sellPrice'] - i['buyPrice']),
            "{:.2f}%".format(((i['sellPrice'] - i['buyPrice']) / i['buyPrice']) * 100),
            memo
        ]

        i['stockPrice'] = 0
        i['id'] = ""
        i['unit'] = ""
        i['buyPrice'] = ""
        i['buyDate'] = ""
        i['buyIndex'] = 9999
        i['stop_loss_price'] = 0

        i['sellPrice'] = 0
        i['sellDate'] = ""

        return stockList, stockSaveList
  return stockList, stockSaveList


def get_stock(stock_id):
  for stock in stockList:
    if stock["id"] == stock_id:
      return stock
  return None

def is_stock(stock_id):
  return any(stock.get("id") == stock_id for stock in stockList)


def isEmpty():
  return any(stock.get("id") == '' for stock in stockList)


###################################################
#程式開始

buymax = 10
maxMoney = 200000

for j in range(buymax):
  i = {
      'balance': maxMoney // buymax,
      'id': '',
      'unit': 0,
      'buyPrice': 0,
      'buyDate': '',
      'sellPrice': 0,
      'sellDate': '',
      'buyIndex': 9999,
      'stop_loss_price':0,
      'stockPrice':0

  }
  stockList.append(i)

buySignal1 = getBuySignal(40,3)
buySignal2 = getBuySignal(30,3)
#buySignal3 = getBuySignal(20,3)
buySignals = buySignal1 | buySignal2 #| buySignal3


# 計算每日漲跌幅（%）
daily_return = close.pct_change() * 100

sellAct = ((daily_return.shift(1)+daily_return.shift(2))>16) | (daily_return.shift(1).rolling(3).sum()>18)  #快速停利狀況

close['type'] = 'close'
sellAct['type'] = 'sellAct'
buySignals['type'] = 'buySignals'



df_combined = pd.concat([close,buySignals,sellAct,daily_return], axis=0)
df_combined = df_combined.set_index('type', append=True).unstack(level=-1)

print(df_combined.shape[0])


allList = close.keys()
allList = allList[:-1]


printList = []

previous_row = None  # 用來保存上一行的資料

for int_index, (date, row) in enumerate(df_combined.iterrows()):  #按日
  for id in allList:
    if(not is_stock(id) and isEmpty()):
      closePrice = row[id]['close']
      if(row[id]['buySignals']==1):
        if(closePrice<previous_row[id]['close']*1.05 and closePrice>previous_row[id]['close']*0.94): #比起昨日漲跌5%以上不接
          buyStock(stockList,id,closePrice,date,int_index)


    elif(is_stock(id)):
      closePrice = row[id]['close']
      nowStock = get_stock(id)

      if(row[id]['sellAct']==1 and int_index>(nowStock["buyIndex"]+3)): #至少持股3天以上
        sellStock(stockSaveList,stockList,id,closePrice,date,'短期暴漲')

      elif(nowStock["stop_loss_price"]>closePrice):
        sellStock(stockSaveList,stockList,id,closePrice,date,'停損')

      elif(closePrice>nowStock["stockPrice"]*1.2): #最大利潤 20%左右
        sellStock(stockSaveList,stockList,id,closePrice,date,'停利，買進最大利潤')

      elif(int_index>(nowStock["buyIndex"]+10) and (closePrice> nowStock["stockPrice"]*1.1)): #持倉超過半個月 且 利潤N%左右

        sellStock(stockSaveList,stockList,id,closePrice,date,'持倉超過半個月，獲得部分利潤')

      elif(int_index>(nowStock["buyIndex"]+20)): #不持倉超過一個月
        sellStock(stockSaveList,stockList,id,closePrice,date,'持倉超過一個月，強迫收攤')

  previous_row = row


print("--------------------------")
for stock in stockList:
  print(stock)

print('--------紀錄--------')
print(stockSaveList)


<ipython-input-33-c24edc8a4ecf>:180: FutureWarning: The default fill_method='pad' in FinlabDataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_return = close.pct_change() * 100


283
--------------------------
{'balance': 21375.35, 'id': '', 'unit': '', 'buyPrice': '', 'buyDate': '', 'sellPrice': 0, 'sellDate': '', 'buyIndex': 9999, 'stop_loss_price': 0, 'stockPrice': 0}
{'balance': 20091.52, 'id': '', 'unit': '', 'buyPrice': '', 'buyDate': '', 'sellPrice': 0, 'sellDate': '', 'buyIndex': 9999, 'stop_loss_price': 0, 'stockPrice': 0}
{'balance': 19816.0, 'id': '', 'unit': '', 'buyPrice': '', 'buyDate': '', 'sellPrice': 0, 'sellDate': '', 'buyIndex': 9999, 'stop_loss_price': 0, 'stockPrice': 0}
{'balance': 20037.25, 'id': '', 'unit': '', 'buyPrice': '', 'buyDate': '', 'sellPrice': 0, 'sellDate': '', 'buyIndex': 9999, 'stop_loss_price': 0, 'stockPrice': 0}
{'balance': 20540.6, 'id': '', 'unit': '', 'buyPrice': '', 'buyDate': '', 'sellPrice': 0, 'sellDate': '', 'buyIndex': 9999, 'stop_loss_price': 0, 'stockPrice': 0}
{'balance': 19348.0, 'id': '', 'unit': '', 'buyPrice': '', 'buyDate': '', 'sellPrice': 0, 'sellDate': '', 'buyIndex': 9999, 'stop_loss_price': 0, 'stoc